In [54]:
from config import Config
import pandas
import os
import m3u8
import datetime
import json

channel="nickjr_rom"
txdate="2019-10-08"
day_start=datetime.time(6,0)
group="NickJR"


In [41]:
def get_commercial_log(channel,txdate):
    breaks=[]
    local_feed=[lf for lf in Config.LOCALIZED_FEEDS if lf["dir"]==channel][0]
    xls_filename=os.path.join(Config.STREAMS_ROOT,local_feed["dir"],"commercials","playlists",
                          txdate,txdate+"_"+local_feed["log"]+".xls")
    df=pandas.read_excel(xls_filename)
    for b in df["Break Number"].unique():
        com_break={}
        com_break["id"]=b
        com_break["starttime"]=df[df["Break Number"] == b].iloc[0]["Start Time"][0:-3]
        com_break["start_time_dt"]=datetime.datetime.strptime(txdate+" "+com_break["starttime"],"%Y-%m-%d %H:%M:%S")
        if datetime.datetime.strptime(com_break["starttime"],"%H:%M:%S").time()<day_start:
            com_break["start_time_dt"]=com_break["start_time_dt"]+datetime.timedelta(days=1)
        com_break["duration"]='{:02d}:{:02d}'.format(int(df[df["Break Number"] == b]["Duartion WORK"].sum()/60),
                                             df[df["Break Number"] == b]["Duartion WORK"].sum()%60)
        com_break["duration_td"]=datetime.timedelta(minutes=int(com_break["duration"].split(':')[0]),
                                                    seconds=int(com_break["duration"].split(':')[1]))
        if os.path.isfile(os.path.join(Config.STREAMS_ROOT,local_feed["dir"],"commercials","playlists",
                                           txdate,'brk_{:02d}'.format(b),"playlist.m3u8")):
            com_break["manifest"]=os.path.join(Config.BASE_URI,local_feed["dir"],"commercials","playlists",
                                           txdate,'brk_{:02d}'.format(b),"playlist.m3u8")
            com_break["segments"]=[]
            for segment in m3u8.load(com_break["manifest"]).segments:
                com_break["segments"].append(segment.uri)
        breaks.append(com_break)
    return breaks

In [42]:
breaks=get_commercial_log(channel,txdate)

In [43]:
breaks

[{'id': 1,
  'starttime': '06:22:00',
  'start_time_dt': datetime.datetime(2019, 10, 8, 6, 22),
  'duration': '06:20',
  'duration_td': datetime.timedelta(seconds=380)},
 {'id': 2,
  'starttime': '06:49:00',
  'start_time_dt': datetime.datetime(2019, 10, 8, 6, 49),
  'duration': '05:40',
  'duration_td': datetime.timedelta(seconds=340)},
 {'id': 3,
  'starttime': '07:17:00',
  'start_time_dt': datetime.datetime(2019, 10, 8, 7, 17),
  'duration': '06:20',
  'duration_td': datetime.timedelta(seconds=380)},
 {'id': 4,
  'starttime': '07:47:00',
  'start_time_dt': datetime.datetime(2019, 10, 8, 7, 47),
  'duration': '05:40',
  'duration_td': datetime.timedelta(seconds=340)},
 {'id': 5,
  'starttime': '08:27:00',
  'start_time_dt': datetime.datetime(2019, 10, 8, 8, 27),
  'duration': '06:20',
  'duration_td': datetime.timedelta(seconds=380)},
 {'id': 6,
  'starttime': '08:54:00',
  'start_time_dt': datetime.datetime(2019, 10, 8, 8, 54),
  'duration': '05:40',
  'duration_td': datetime.timed

In [59]:
def get_timeline(txdate):
    channels={"Nickelodeon":"nick_hun",
              "Nick JR":"nickjr_hun",
              "Nicktoons":"nicktoons_hun"}
    events=[]
    timeline={}
    timeline["title"]={"text":{"text":"Commercial breaks","headline":txdate}}
    for group,channel in channels.items():
        breaks=get_commercial_log(channel,txdate)
        for b in breaks:
            event={}
            event["start_date"]={"year":b["start_time_dt"].year,"month":b["start_time_dt"].month,"day":b["start_time_dt"].day,
                                 "hour":b["start_time_dt"].hour,"minute":b["start_time_dt"].minute,"second":b["start_time_dt"].second}
            end_date=b["start_time_dt"]+b["duration_td"]
            event["end_date"]={"year":end_date.year,"month":end_date.month,"day":end_date.day,
                                 "hour":end_date.hour,"minute":end_date.minute,"second":end_date.second}
            event["text"]={"text":"Break {:02d}".format(b["id"]),"headline":""}
            event["group"]=group
            event["background"]={"color":"#0f9bd1"}
            events.append(event)
        timeline["events"]=events

    return json.dumps(timeline)
    

In [60]:
print(get_timeline(txdate))

{"title": {"text": {"text": "Commercial breaks", "headline": "2019-10-08"}}, "events": [{"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 16, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 22, "second": 10}, "text": {"text": "Break 01", "headline": ""}, "group": "Nickelodeon", "background": {"color": "#0f9bd1"}}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 44, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 50, "second": 10}, "text": {"text": "Break 02", "headline": ""}, "group": "Nickelodeon", "background": {"color": "#0f9bd1"}}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 28, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 34, "second": 10}, "text": {"text": "Break 03", "headline": ""}, "group": "Nickelodeon", "background": {"color": "#0f9bd1"}}, {"start_date": {"year": 2019, "month": 10, "d

{"title": 
   {"text": 
      {"text": "Commercial breaks", "headline": "2019-10-08"}
   }, 
 "events": [
    {
    "start_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 16, "second": 0}, 
    "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 22, "second": 10}, 
    "text": {"text": "Break 01", "headline": ""}, 
    "group": "Nickelodeon", 
    "background": "#0f9bd1"
    },
    
    {
    "start_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 44, "second": 0}, 
    "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 50, "second": 10}, "text": {"text": "Break 02", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 28, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 34, "second": 10}, "text": {"text": "Break 03", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 52, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 58, "second": 10}, "text": {"text": "Break 04", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 14, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 20, "second": 10}, "text": {"text": "Break 05", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 42, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 48, "second": 10}, "text": {"text": "Break 06", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 10, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 16, "second": 10}, "text": {"text": "Break 07", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 37, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 43, "second": 10}, "text": {"text": "Break 08", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 6, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 12, "second": 10}, "text": {"text": "Break 09", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 33, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 39, "second": 10}, "text": {"text": "Break 10", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 24, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 30, "second": 10}, "text": {"text": "Break 11", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 42, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 48, "second": 10}, "text": {"text": "Break 12", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 12, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 18, "second": 10}, "text": {"text": "Break 13", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 40, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 46, "second": 10}, "text": {"text": "Break 14", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 8, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 14, "second": 10}, "text": {"text": "Break 15", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 36, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 42, "second": 10}, "text": {"text": "Break 16", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 4, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 10, "second": 10}, "text": {"text": "Break 17", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 32, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 38, "second": 10}, "text": {"text": "Break 18", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 20, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 26, "second": 10}, "text": {"text": "Break 19", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 48, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 54, "second": 10}, "text": {"text": "Break 20", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 9, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 15, "second": 10}, "text": {"text": "Break 21", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 42, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 48, "second": 10}, "text": {"text": "Break 22", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 9, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 15, "second": 10}, "text": {"text": "Break 23", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 37, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 43, "second": 10}, "text": {"text": "Break 24", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 7, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 13, "second": 10}, "text": {"text": "Break 25", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 35, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 41, "second": 10}, "text": {"text": "Break 26", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 23, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 29, "second": 10}, "text": {"text": "Break 27", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 51, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 57, "second": 10}, "text": {"text": "Break 28", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 20, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 26, "second": 10}, "text": {"text": "Break 29", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 47, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 53, "second": 10}, "text": {"text": "Break 30", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 17, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 23, "second": 10}, "text": {"text": "Break 31", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 45, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 51, "second": 10}, "text": {"text": "Break 32", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 13, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 19, "second": 10}, "text": {"text": "Break 33", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 43, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 49, "second": 10}, "text": {"text": "Break 34", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 11, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 17, "second": 10}, "text": {"text": "Break 35", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 39, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 45, "second": 10}, "text": {"text": "Break 36", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 0, "minute": 8, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 0, "minute": 14, "second": 10}, "text": {"text": "Break 37", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 0, "minute": 32, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 0, "minute": 38, "second": 10}, "text": {"text": "Break 38", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 1, "minute": 0, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 1, "minute": 6, "second": 10}, "text": {"text": "Break 39", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 1, "minute": 29, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 1, "minute": 35, "second": 10}, "text": {"text": "Break 40", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 4, "minute": 12, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 4, "minute": 18, "second": 10}, "text": {"text": "Break 41", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 4, "minute": 31, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 4, "minute": 37, "second": 10}, "text": {"text": "Break 42", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 5, "minute": 12, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 5, "minute": 18, "second": 10}, "text": {"text": "Break 43", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 5, "minute": 29, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 5, "minute": 35, "second": 10}, "text": {"text": "Break 44", "headline": ""}, "group": "Nickelodeon", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 22, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 28, "second": 20}, "text": {"text": "Break 01", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 49, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 54, "second": 40}, "text": {"text": "Break 02", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 17, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 23, "second": 20}, "text": {"text": "Break 03", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 47, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 52, "second": 40}, "text": {"text": "Break 04", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 27, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 33, "second": 20}, "text": {"text": "Break 05", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 54, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 59, "second": 40}, "text": {"text": "Break 06", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 22, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 28, "second": 20}, "text": {"text": "Break 07", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 50, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 55, "second": 40}, "text": {"text": "Break 08", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 18, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 31, "second": 25}, "text": {"text": "Break 09", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 55, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 59, "second": 40}, "text": {"text": "Break 10", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 21, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 27, "second": 20}, "text": {"text": "Break 11", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 48, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 53, "second": 40}, "text": {"text": "Break 12", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 16, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 22, "second": 20}, "text": {"text": "Break 13", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 44, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 49, "second": 40}, "text": {"text": "Break 14", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 17, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 23, "second": 20}, "text": {"text": "Break 15", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 45, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 50, "second": 40}, "text": {"text": "Break 16", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 14, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 19, "second": 40}, "text": {"text": "Break 17", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 32, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 38, "second": 20}, "text": {"text": "Break 18", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 14, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 20, "second": 20}, "text": {"text": "Break 19", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 25, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 30, "second": 40}, "text": {"text": "Break 20", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 26, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 32, "second": 20}, "text": {"text": "Break 21", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 43, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 48, "second": 40}, "text": {"text": "Break 22", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 12, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 17, "second": 40}, "text": {"text": "Break 23", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 40, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 46, "second": 20}, "text": {"text": "Break 24", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 7, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 13, "second": 20}, "text": {"text": "Break 25", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 36, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 41, "second": 40}, "text": {"text": "Break 26", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 4, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 9, "second": 40}, "text": {"text": "Break 27", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 32, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 38, "second": 20}, "text": {"text": "Break 28", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 2, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 15, "second": 25}, "text": {"text": "Break 29", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 38, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 42, "second": 40}, "text": {"text": "Break 30", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 21, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 26, "second": 40}, "text": {"text": "Break 31", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 49, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 55, "second": 20}, "text": {"text": "Break 32", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 18, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 23, "second": 40}, "text": {"text": "Break 33", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 44, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 50, "second": 20}, "text": {"text": "Break 34", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 11, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 17, "second": 20}, "text": {"text": "Break 35", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 40, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 45, "second": 40}, "text": {"text": "Break 36", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 0, "minute": 20, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 0, "minute": 26, "second": 20}, "text": {"text": "Break 37", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 0, "minute": 48, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 0, "minute": 53, "second": 40}, "text": {"text": "Break 38", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 1, "minute": 17, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 1, "minute": 23, "second": 20}, "text": {"text": "Break 39", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 9, "hour": 1, "minute": 45, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 9, "hour": 1, "minute": 50, "second": 40}, "text": {"text": "Break 40", "headline": ""}, "group": "Nick JR", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 6, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 11, "second": 40}, "text": {"text": "Break 01", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 34, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 6, "minute": 40, "second": 20}, "text": {"text": "Break 02", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 25, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 31, "second": 20}, "text": {"text": "Break 03", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 54, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 7, "minute": 59, "second": 40}, "text": {"text": "Break 04", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 21, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 26, "second": 40}, "text": {"text": "Break 05", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 49, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 8, "minute": 55, "second": 20}, "text": {"text": "Break 06", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 16, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 21, "second": 40}, "text": {"text": "Break 07", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 43, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 9, "minute": 49, "second": 20}, "text": {"text": "Break 08", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 23, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 29, "second": 20}, "text": {"text": "Break 09", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 52, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 10, "minute": 57, "second": 40}, "text": {"text": "Break 10", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 21, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 26, "second": 40}, "text": {"text": "Break 11", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 49, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 11, "minute": 55, "second": 20}, "text": {"text": "Break 12", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 19, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 24, "second": 40}, "text": {"text": "Break 13", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 46, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 12, "minute": 52, "second": 20}, "text": {"text": "Break 14", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 14, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 20, "second": 20}, "text": {"text": "Break 15", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 43, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 13, "minute": 48, "second": 40}, "text": {"text": "Break 16", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 12, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 18, "second": 20}, "text": {"text": "Break 17", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 41, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 14, "minute": 46, "second": 40}, "text": {"text": "Break 18", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 8, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 14, "second": 20}, "text": {"text": "Break 19", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 45, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 15, "minute": 50, "second": 40}, "text": {"text": "Break 20", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 24, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 30, "second": 20}, "text": {"text": "Break 21", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 52, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 16, "minute": 57, "second": 40}, "text": {"text": "Break 22", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 20, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 25, "second": 40}, "text": {"text": "Break 23", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 49, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 17, "minute": 55, "second": 20}, "text": {"text": "Break 24", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 16, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 22, "second": 20}, "text": {"text": "Break 25", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 45, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 18, "minute": 50, "second": 40}, "text": {"text": "Break 26", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 12, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 17, "second": 40}, "text": {"text": "Break 27", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 29, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 19, "minute": 35, "second": 20}, "text": {"text": "Break 28", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 22, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 28, "second": 20}, "text": {"text": "Break 29", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 50, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 20, "minute": 55, "second": 40}, "text": {"text": "Break 30", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 18, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 23, "second": 40}, "text": {"text": "Break 31", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 45, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 21, "minute": 51, "second": 20}, "text": {"text": "Break 32", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 14, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 20, "second": 20}, "text": {"text": "Break 33", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 43, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 22, "minute": 48, "second": 40}, "text": {"text": "Break 34", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, {"start_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 11, "second": 0}, "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 16, "second": 40}, "text": {"text": "Break 35", "headline": ""}, "group": "Nicktoons", "background": "#0f9bd1"}, 
    {
    "start_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 39, "second": 0}, 
    "end_date": {"year": 2019, "month": 10, "day": 8, "hour": 23, "minute": 45, "second": 20}, 
    "text": {"text": "Break 36", "headline": ""}, 
    "group": "Nicktoons", 
    "background": "#0f9bd1"}
  ]
}